# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym==0.29.1
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium==0.29.1

In [3]:
# %pip install stable-baselines3 plotly numpy pandas
# %pip install ipywidgets
# %pip install pandas_ta

In [4]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [5]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import torch
import fnmatch

In [6]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [7]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [8]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [9]:
import plotly.express as px
import plotly.graph_objects as go

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA доступна. Работаем на GPU.


# Open Data

## Description of changes

Основано на Model 2, Отличия:
- Подаем на вход 3 дополнительные фичи, нормализованные делением на их максимальные значения:
    - tokens_held / 50000000
    - balance / 100000
    - net_worth / 100000

## Settings

In [11]:
window_length = 48
nb_steps = 80000

model_num = 10
data_num = 1

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

## Open Data

In [12]:
def find_dataset_file(dnum, folder="data"):
    pattern = f"dataset_{dnum}D_*.csv"
    for filename in os.listdir(folder):
        if fnmatch.fnmatch(filename, pattern):
            return os.path.join(folder, filename)
    return None

In [13]:
data_path = find_dataset_file(data_num, folder= Path["processed"])
print(data_path)
df = pd.read_csv(data_path)
df.shape

d:\PythonScripts\RL_for_Trading\data\processed\dataset_1D_Standart_26cols_26tkn_1t.csv


(103954, 26)

In [14]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [15]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [16]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [17]:
def split_by_asset_ids(df: pd.DataFrame, test_asset_ids: list):
    test_df = df[df['Asset_ID_encoded'].isin(test_asset_ids)]
    train_df = df[~df['Asset_ID_encoded'].isin(test_asset_ids)]
    return train_df, test_df

In [18]:
test_asset_ids = [21, 22, 23, 24, 25]
train_df, test_df = split_by_asset_ids(df = df, test_asset_ids = test_asset_ids)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83280, 26)
Testing data shape: (20674, 26)


# Game Rule

In [19]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df, mode):
        super(TradingEnv, self).__init__()

        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length
        self.mode = mode # test or train

        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        num_features = len(self.df.columns) - 1 + 3 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0
        self.go_beyond = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        print(self.now_token)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'tokens_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
            "cnt_go_beyond":[],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {asset_ids[0]:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            if self.mode == "train":
                self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset
            else:
                return None, None

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'tokens_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
                "cnt_go_beyond":[],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]

        # Нормализуем текущие значения переменных на каждом шаге
        tokens_held_scaled = self.tokens_held / 50000000
        balance_scaled = self.balance / 100000
        net_worth_scaled = self.net_worth / 100000

        frame['tokens_held'] = tokens_held_scaled
        frame['balance'] = balance_scaled
        frame['net_worth'] = net_worth_scaled

        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        terminated = False
        truncated = False
        self.prev_token = self.now_token

        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.current_step += 1  # Move to the next time step
        
        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                self.now_token -=1
                terminated = True

        if not terminated:
            #  Теперь мы будем награждать модель используя данные за следующий шаг, а не предыдущий
            self.future_price = self.df.loc[self.current_step, 'Close_orig'] 
            self.fut_net_worth = self.balance + self.tokens_held * self.future_price

            if self.fut_net_worth != 0:
                net_worth_change = (self.fut_net_worth * 100 / self.net_worth) - 100
                price_change = (self.future_price * 100 / self.current_price) - 100
                initial_change = self.net_worth / self.initial_balance
                
                if price_change > 0:
                    self.reward += net_worth_change - (abs(abs(net_worth_change) - abs(price_change)) / 2)
                else:
                    self.reward += net_worth_change + (abs(abs(net_worth_change) - abs(price_change)) / 2)

                if self.net_worth > self.initial_balance:
                    self.reward += initial_change
                else:
                    self.reward += - (1 - initial_change)
                
                self.logger.info(f"step: {self.current_price - 1}, net_worth_change: {net_worth_change}, price_change: {price_change}, initial_change: {initial_change}, reward: {self.reward}")

                if self.net_worth < self.initial_balance * 0.5: # Только во время тренировки штрафуем за проеб половины баланса
                    if self.mode == "train":
                        # self.reward += -50 
                        self.logger.info("Net worth dropped below 50% of initial balance.")
                        terminated = True
            else:
                self.logger.info("fut_net_worth == 0")

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step - 1)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["tokens_held"].append(self.tokens_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)
        self.hist["cnt_go_beyond"].append(self.go_beyond)

        return obs, self.reward, terminated, truncated, info


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.tokens_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.tokens_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.tokens_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.go_beyond = 1
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.tokens_held:
            shares_to_sell = self.tokens_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.tokens_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.go_beyond = -1
            self.logger.info("No shares to sell.")


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.tokens_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [20]:
train_env = TradingEnv(train_df, mode = "train")

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}
-1


In [21]:
test_env = TradingEnv(test_df, mode = "test")

{21: 0, 22: 4093, 23: 8312, 24: 12700, 25: 16302}
20


In [22]:
# check_env(train_env)

# Visualisaton Functions

In [23]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "close_by_asset"))

In [24]:
def plot_reward_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['reward'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['reward'], mode='lines', name='Reward'))

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Reward = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Reward and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Reward')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "reward_by_asset"))

In [25]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "price_change_by_asset"))

In [26]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "token_data"))

In [27]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "action_counts"))

In [28]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "relative_change_by_token"))

# Train Model

## Settings

In [29]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [30]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model_save"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [31]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        
    # callback=eval_callback   
    progress_bar= True,
)

Logging to ./sac_tensorboard/SAC_19


Output()

In [32]:
model.save(Path["model_save"](model_num, data_num))

In [33]:
train_hist = train_env.hist
train_hist_df = pd.DataFrame(train_hist)
print(len(train_hist["action"]))

80000


In [34]:
unique_values = train_hist_df['token'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

## Plots

In [35]:
token = 20
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [36]:
train_hist_df.describe()

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
count,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,80000.000000,8.000000e+04,8.000000e+04,80000.000000
mean,36256.575975,8.387926e+02,1565.458955,1.616049e+08,8.731025,6.860478e+00,-1.072725,-0.023608,1.889742e+11,1.490886e+05,-0.077037
std,23842.321534,1.617778e+03,1962.963598,1.465603e+09,6.114221,1.745757e+01,3.846603,0.731390,1.121970e+12,2.044010e+05,0.994254
min,48.000000,7.006020e-09,474.119048,0.000000e+00,0.000000,5.100000e-08,-93.817499,-1.000000,0.000000e+00,0.000000e+00,-1.000000
25%,13960.000000,1.491303e+01,766.893895,2.000000e+00,3.000000,1.294940e-04,-3.960493,-0.769992,3.146900e+04,3.453750e+04,-1.000000
50%,32827.500000,5.469296e+02,1021.966643,8.230000e+02,8.000000,7.074078e-02,-0.510824,-0.074142,4.416435e+05,8.333904e+04,-1.000000
75%,56717.250000,9.969634e+02,1538.759424,8.340125e+05,14.000000,2.340869e+00,1.009553,0.747773,1.437066e+09,1.762108e+05,1.000000
max,81661.000000,2.417649e+04,24187.923715,3.644917e+10,20.000000,1.065761e+02,254.973291,1.000000,1.112603e+13,1.533600e+06,1.000000


In [37]:
train_hist_df

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
0,48,452.087899,999.452635,9568979,0,0.000057,0.750335,0.547912,0,0.000000,0
1,49,520.493723,1012.168317,8399238,0,0.000059,0.923894,-0.122243,1169741,68.405824,0
2,50,147.197833,1009.897166,14794542,0,0.000058,-0.590035,0.717196,1169741,68.405824,0
3,51,89.093562,992.174437,15810793,0,0.000057,-1.024353,0.394736,1169741,68.405824,0
4,52,812.739770,980.841024,2978037,0,0.000056,-1.110971,-0.811645,14002497,792.052032,0
...,...,...,...,...,...,...,...,...,...,...,...
79995,32177,1049.048970,1049.049111,1,8,0.000141,-4.914071,-0.058315,6784799,1049.048793,-1
79996,32178,843.496453,1048.843767,1428841,8,0.000144,1.245525,0.195942,6784799,1049.048793,-1
79997,32179,1024.002377,1046.705568,159493,8,0.000142,1.893322,-0.888376,8054147,1229.554717,-1
79998,32180,1043.609951,1046.274768,19066,8,0.000140,0.566667,-0.880462,8194574,1249.162290,-1


# Test Model

## Test

In [38]:
model = SAC.load(Path["model_save"](model_num, data_num))

In [39]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist
        if info is None:
            break

In [40]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20433


In [41]:
test_hist_df

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
0,48,321.192759,999.321871,29734680,21,0.000023,-1.313214,0.678807,0,0.000000,0
1,49,6.801309,981.821148,43864488,21,0.000022,-0.000371,0.978825,0,0.000000,0
2,50,0.034465,981.989846,44168558,21,0.000022,-0.692678,0.994934,0,0.000000,0
3,51,0.000380,975.364529,44170100,21,0.000022,0.672765,0.989114,0,0.000000,0
4,52,0.000024,982.166724,44170116,21,0.000022,-0.876801,0.979741,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
20428,20668,511.000340,1180.059849,1,25,669.059508,-3.686183,0.986402,5,1600.892207,1
20429,20669,511.000340,1182.613414,1,25,671.613074,-3.970786,0.987464,5,1600.892207,1
20430,20670,511.000340,1179.685397,1,25,668.685056,-3.966619,0.979125,5,1600.892207,1
20431,20671,511.000340,1176.892199,1,25,665.891859,-3.795771,0.944583,5,1600.892207,1


In [42]:
unique_values = test_hist_df['token'].unique()
unique_values

array([21, 22, 23, 24, 25], dtype=int64)

## Plots

In [43]:
token = 25

In [44]:
plot_token_data(df = test_hist_df, token = token)

In [45]:
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [46]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - hist_df: DataFrame containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall averages for each column
    averages = report_df.mean(numeric_only=True)
    averages['Asset_ID'] = 'Average'  # Mark row as average

    # Append averages row to the DataFrame using pd.concat
    report_df = pd.concat([report_df, pd.DataFrame([averages])], ignore_index=True)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

- Asset_ID: Уникальный идентификатор актива (из столбца token), для которого рассчитываются метрики.

- Total Profit: Общий финансовый результат (прибыль или убыток) по данному активу. Рассчитывается как разница между конечной чистой стоимостью (net_worth) и начальным балансом (initial_balance).

- ROI (%): Доходность инвестиций (Return on Investment) в процентах. Показывает процентный прирост (или убыток) от начальной суммы баланса.

- Sharpe Ratio: Коэффициент Шарпа. Оценивает отношение доходности к риску (волатильности). Чем выше коэффициент Шарпа, тем лучше риск-корректированная доходность стратегии.

- Max Drawdown (%): Максимальная просадка в процентах. Это максимальное снижение стоимости актива от его исторического максимума. Отражает риски стратегии, связанные с падением стоимости.

- Win Rate (%): Процент прибыльных сделок. Это отношение количества прибыльных сделок к общему количеству сделок по активу, умноженное на 100.

- Buy-and-Hold Profit: Прибыль при стратегии "купить и держать". Показывает, сколько можно было бы заработать, если просто купить актив в начале и держать его до конца периода тестирования.

- Buy-and-Hold ROI (%): Доходность при стратегии "купить и держать". Процентный прирост от начальной цены актива, если его просто держать до конца периода.

- Ideal Profit: Идеальная прибыль. Это гипотетическая максимальная прибыль, которую можно было бы получить, если бы купили актив по минимальной цене и продали по максимальной цене за период.

- Ideal ROI (%): Идеальная доходность. Процентный прирост при идеальной стратегии, где покупка происходит по минимальной цене, а продажа — по максимальной.

- Asset Price Change (%): Изменение цены актива в процентах за период. Это процентное изменение цены от начальной до конечной за период тестирования.

In [47]:
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_15108\3864715686.py:85: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [48]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,21,-349.613458,-34.961346,-0.314073,-48.080956,46.427689,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
1,22,17276.008240,1727.600824,0.716348,-35.156953,49.100935,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
2,23,-397.721441,-39.772144,-0.014424,-60.613081,49.285714,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
3,24,-20.225471,-2.022547,0.085586,-43.981879,50.647158,594.470161,59.447016,2103.489180,210.348918,59.447016
4,25,177.414224,17.741422,0.147227,-42.032000,49.572056,233.832793,23.383279,2790.670479,279.067048,23.383279
5,Average,3337.172419,333.717242,0.124133,-45.972974,49.006710,50839.685851,5083.968585,61914.686764,6191.468676,5083.968585


In [49]:
test_hist_df.describe()

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
count,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000,20433.000000
mean,10359.856702,728.171606,1076.307858,1.000390e+06,22.997015,78.111095,-3.769626,-0.556462,1.842241e+08,20213.756118,-0.637988
std,5965.411689,502.674903,1208.094934,5.187597e+06,1.419961,154.761422,2.814882,0.637353,4.051419e+08,15135.407688,0.769907
min,48.000000,0.000004,527.887335,1.000000e+00,21.000000,0.000011,-31.205390,-0.999994,0.000000e+00,0.000000,-1.000000
25%,5204.000000,410.519401,631.132794,1.000000e+00,22.000000,0.008798,-5.359871,-0.979865,3.646000e+03,4998.912164,-1.000000
50%,10360.000000,581.973418,807.947769,1.000000e+00,23.000000,0.272110,-4.738362,-0.901581,4.602500e+04,21140.543514,-1.000000
75%,15516.000000,1197.313225,1233.853693,1.000000e+00,24.000000,8.408479,-2.994482,-0.396964,5.863154e+06,33960.945204,-1.000000
max,20672.000000,3719.785831,21116.005819,4.536160e+07,25.000000,674.820251,44.335041,0.999994,1.502299e+09,62729.082066,1.000000


In [50]:
report_df.to_csv(Path["reports"](model_num, data_num, "test"), index= False)
test_hist_df.describe().to_csv(Path["reports"](model_num, data_num, "test_describe"), index= False)

In [51]:
overall_metrics

{'Total Profit': 20023.034511724658,
 'ROI (%)': 400.4606902344932,
 'Sharpe Ratio': 0.12413268396168102,
 'Win Rate (%)': 49.006710478316,
 'Buy-and-Hold Profit': 305038.1151049347,
 'Buy-and-Hold ROI (%)': 6100.762302098695}